In [2]:
import pandas as pd 
import re
from gensim.models import doc2vec,ldamodel
from gensim import corpora,similarities
import numpy as np

In [3]:
def clean_email_text(text):
    # 数据清洗
    text = text.replace('\n', " ")  # 新行，我们是不需要的
    text = re.sub(r"-", " ", text)  # 把 "-" 的两个单词，分开。（比如：july-edu ==> july edu）
    text = re.sub(r"\d+/\d+/\d+", "", text)  # 日期，对主体模型没什么意义
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text)  # 时间，没意义
    text = re.sub(r"[\w]+@[\.\w]+", "", text)  # 邮件地址，没意义
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text)  # 网址，没意义
    pure_text = ''
    # 以防还有其他特殊字符（数字）等等，我们直接把他们loop一遍，过滤掉
    for letter in text:
        # 只留下字母和空格
        if letter.isalpha() or letter == ' ':
            pure_text += letter
    # 再把那些去除特殊字符后落单的单词，直接排除。
    # 我们就只剩下有意义的单词了。
    text = ' '.join(word for word in pure_text.split() if len(word) > 1)  # 而且单词长度必须是2以上
    return text

In [4]:
def stopwords():
    stopwords = []
    with open('stopwords.txt','r',encoding='utf-8') as f:
        stopwords = f.read().split()
    return stopwords   

In [5]:
if __name__ == '__main__':
    df = pd.read_csv('Emails.csv')
    df = df[['Id','ExtractedBodyText']].dropna()

    docs =df['ExtractedBodyText']
    docs = docs.apply(lambda s:clean_email_text(s))
    doclist = docs.values
    
    dellist = stopwords()
    texts = [[word for word in doc.lower().split() if word not in dellist] for doc in doclist]
    
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    
    lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20,random_state=None,passes=10)
    
    lda.save('samuellda.model')
    
    lda = ldamodel.LdaModel.load('samuellda.model')
    text1 = 'I was greeted by this heartwarming display on the corner of my street today. ' \
           'Thank you to all of you who did this. Happy Thanksgiving. -H'
    text1 = clean_email_text(text1)
    texts1 = [word for word in text1.lower().split() if word not in dellist]
    bow1 = dictionary.doc2bow(texts1)
    doc_lda1 = lda[bow1]
    list_doc1 = [i[1] for i in doc_lda1]

#     print(lda.get_document_topics(bow1))
#     doc_lda1 = lda[bow1]
#     print(doc_lda1)

    text2 = 'I always love to greet with my friends at the corner of the street.'
    text2 = clean_email_text(text2)
    texts2 = [word for word in text2.lower().split() if word not in dellist]
    bow2 = dictionary.doc2bow(texts2)
    doc_lda2 = lda[bow2]
    list_doc2 = [i[1] for i in doc_lda2]
    
    try:
        sim = np.dot(list_doc1, list_doc2) / (np.linalg.norm(list_doc1) * np.linalg.norm(list_doc2))
    except ValueError:
        sim=0

    print(doc_lda1)
#     with open('Emails.csv','r',encoding='utf-8') as p:
#         txt=p.read()
#     print(txt.lower().split())
#     txt = [word for word in txt.lower().split() if word not in dellist]

[(14, 0.23996672), (16, 0.25371647), (18, 0.3645104)]


In [19]:
import numpy as np
list = [1.2,0.2,0.3,0.4,0.5,0.6]
list1 = [1,0.5,0.5,0.2,0.3,0.6]
array = np.array(list)
array1 = np.array(list1)
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim
print(cos_sim(list,list1))
print(array)

0.972678343474334
[1.2 0.2 0.3 0.4 0.5 0.6]


In [7]:
##情绪词语列表整理
Happy = []
Good = []
Surprise = []
Anger = []
Sad = []
Fear = []
Disgust = []
for idx,row in df.iterrows():
    if row['情感分类'] in ['PA','PE']:
        Happy.append(row['词语'])
    if row['情感分类'] in ['PD','PH','PG','PB','PK']:
        Good.append(row['词语'])
    if row['情感分类'] in ['PC']:
        Surprise.append(row['词语'])
    if row['情感分类'] in ['NA']:
        Anger.append(row['词语'])
    if row['情感分类'] in ['NB','NJ','NH','PF']:
        Sad.append(row['词语'])
    if row['情感分类'] in ['NI','NC','NG']:
        Fear.append(row['词语'])
    if row['情感分类'] in ['NE','ND','NN','NK','NL']:
        Disgust.append(row['词语'])
Positive = Happy+Good+Surprise
Negative = Anger+Sad+Fear+Disgust
print('well_done')

NameError: name 'df' is not defined